In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [ ]:

headers = {
    'authority': 'www.amazon.in',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
}


In [ ]:

search_query="nike+shoes+men"
base_url="https://www.amazon.in/s?k="
url=base_url+search_query
url


In [ ]:



search_response=requests.get(url,headers=headers)




In [ ]:





search_response.status_code



In [ ]:



cookie={} # insert request cookies within{}
def getAmazonSearch(search_query):
    url="https://www.amazon.in/s?k="+search_query
    print(url)
    page=requests.get(url,headers=headers)
    if page.status_code==200:
        return page
    else:
        return "Error"


In [ ]:

def Searchasin(asin):
    url="https://www.amazon.in/dp/"+asin
    print(url)
    page=requests.get(url,cookies=cookie,headers=headers)
    if page.status_code==200:
        return page
    else:
        return "Error"


In [ ]:

def Searchreviews(review_link):
    url="https://www.amazon.in"+review_link
    print(url)
    page=requests.get(url,cookies=cookie,headers=headers)
    if page.status_code==200:
        return page
    else:
        return "Error"


In [ ]:

product_names=[]
response=getAmazonSearch('nike+shoes+men')
soup=BeautifulSoup(response.content)
for i in soup.findAll("span",{'class':'a-size-base-plus a-color-base a-text-normal'}): # the tag which is common for all the names of products
    product_names.append(i.text) 


In [ ]:


data_asin=[]

response=getAmazonSearch('nike+shoes+men')
soup=BeautifulSoup(response.content)
for i in soup.findAll("div",{'class':"sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20"}):
    data_asin.append(i['data-asin'])
    


In [ ]:

link=[]
for i in range(len(data_asin)):
    response=Searchasin(data_asin[i])
    soup=BeautifulSoup(response.content)
    for i in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
        link.append(i['href'])


In [ ]:



reviews = []

for j in range(len(link)):
    for k in range(10):
        response = Searchreviews(link[j] + '&pageNumber=' + str(k))
        soup = BeautifulSoup(response.content)
        for i in soup.findAll("div", {'data-hook': "review"}):
            try:
                name = i.select_one('[class="a-profile-name"]').text.strip()
            except Exception as e:
                name = 'N/A'

            try:
                stars = i.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
            except Exception as e:
                stars = 'N/A'   

            try:
                title = i.select_one('[data-hook="review-title"]').text.strip().split('\n')[1]
            except Exception as e:
                title = 'N/A'

            try:
                # Convert date str to dd/mm/yyy format
                date = i.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]

            except Exception as e:
                date = 'N/A'

            try:
                description = i.select_one('[data-hook="review-body"]').text.strip()
            except Exception as e:
                description = 'N/A'
                
            try:
                verification = i.select_one('[data-hook="avp-badge"]').text.strip()
            except Exception as e:
                verification = 'N/A'
                
            # create Dictionary with all review data 
            data_dict = {
                'product':product_names[j],
                'Name': name,
                'Stars': stars,
                'Title': title,
                'Date': date,
                'Verification': verification,
                'Description': description
            }

            # Add Dictionary in master empty List
            reviews.append(data_dict)



In [ ]:

review_data=pd.DataFrame.from_dict(reviews)
pd.set_option('max_colwidth',800)


In [ ]:

review_data.to_csv('Scraping reviews.csv')